In [2]:
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('fivethirtyeight')
plt.ion()

import warnings
warnings.filterwarnings('ignore')

In [3]:
import easydict
args = easydict.EasyDict() #dict values 에 .으로 접근 가능하다 

# path 정보
args.default_path = 'C:\\Users\\User\\Desktop\\' # 파일 위치 
args.train_csv = args.default_path+'train.csv'
args.test_csv = args.default_path+'test.csv'
args.default_submission_csv = args.default_path+'submission.csv'

args.submission_csv = args.default_path+'submission_0331.csv' # 새로운 파일 
args.save_results = args.default_path+"model_results.csv" # 정보저장 파일 

# 데이터 분석을 위한 변수들
args.random_state = 21
args.results = []

args.save_results

'C:\\Users\\User\\Desktop\\model_results.csv'

### 데이터 로드

In [4]:
ori_train = pd.read_csv(args.train_csv)
ori_test = pd.read_csv(args.test_csv)
ori_train.shape, ori_test.shape

((916, 12), (393, 11))

### 폰트 설정

In [5]:
import matplotlib as mpl
# Add every font at the specified location
font_dir = ['./fonts']
for font in mpl.font_manager.findSystemFonts(fontpaths=font_dir):
 print(mpl.font_manager.FontProperties(fname=font).get_name())
 mpl.font_manager.fontManager.addfont(font)

In [6]:
# 시스템 폰트 경로 리스트
font_list = mpl.font_manager.findSystemFonts()
print(font_list)
# 폰트 설정하기
prop = mpl.font_manager.FontProperties(fname='원하는 폰트 경로')
plt.rcParams['font.family'] = prop.get_family()

['C:\\Windows\\Fonts\\segoeui.ttf', 'C:\\Windows\\Fonts\\seguisli.ttf', 'C:\\Windows\\Fonts\\segoeuil.ttf', 'C:\\Windows\\Fonts\\seguibl.ttf', 'C:\\Windows\\Fonts\\H2GSRB.TTF', 'C:\\Windows\\Fonts\\segoeuib.ttf', 'C:\\Windows\\Fonts\\NirmalaS.ttf', 'C:\\Windows\\Fonts\\gulim.ttc', 'C:\\Windows\\Fonts\\REFSAN.TTF', 'C:\\Windows\\Fonts\\WINGDNG3.TTF', 'C:\\Windows\\Fonts\\segoeuiz.ttf', 'C:\\Windows\\Fonts\\wingding.ttf', 'C:\\Windows\\Fonts\\arialbd.ttf', 'C:\\Windows\\Fonts\\HancomHoonminjeongeumH.ttf', 'C:\\Windows\\Fonts\\msyhbd.ttc', 'C:\\Windows\\Fonts\\WINGDNG2.TTF', 'C:\\Windows\\Fonts\\comicz.ttf', 'C:\\Windows\\Fonts\\framdit.ttf', 'C:\\Windows\\Fonts\\H2SA1M.TTF', 'C:\\Windows\\Fonts\\Hancom Gothic Bold.ttf', 'C:\\Windows\\Fonts\\timesbd.ttf', 'C:\\Windows\\Fonts\\georgia.ttf', 'C:\\Windows\\Fonts\\monbaiti.ttf', 'C:\\Windows\\Fonts\\cambriaz.ttf', 'C:\\Windows\\Fonts\\msgothic.ttc', 'C:\\Windows\\Fonts\\DUBAI-BOLD.TTF', 'C:\\Windows\\Fonts\\ANTQUAI.TTF', 'C:\\Windows\\Fonts\\

In [7]:
# Set font family globally
mpl.rcParams['font.family'] = 'Noto Sans KR'
print(mpl.rcParams['font.family'])

['Noto Sans KR']


# 데이터 확인 

In [8]:
ori_train.columns

Index(['passengerid', 'survived', 'pclass', 'name', 'gender', 'age', 'sibsp',
       'parch', 'ticket', 'fare', 'cabin', 'embarked'],
      dtype='object')

In [9]:
ori_train.head(2)

,passengerid,survived,pclass,name,gender,age,sibsp,parch,ticket,fare,cabin,embarked
0,0,0,2,"Wheeler, Mr. Edwin Frederick""""",male,NaN,0,0,SC/PARIS 2159,12.875,NaN,S
1,1,0,3,"Henry, Miss. Delia",female,NaN,0,0,382649,7.750,NaN,Q


In [10]:
ori_test.columns # survived 가 빠짐 

Index(['passengerid', 'pclass', 'name', 'gender', 'age', 'sibsp', 'parch',
       'ticket', 'fare', 'cabin', 'embarked'],
      dtype='object')

In [11]:
ori_test.head(2)

,passengerid,pclass,name,gender,age,sibsp,parch,ticket,fare,cabin,embarked
0,916,3,"McGowan, Miss. Anna ""Annie""",female,15.0,0,0,330923,8.0292,NaN,Q
1,917,2,"Pinsky, Mrs. (Rosa)",female,32.0,0,0,234604,13.0000,NaN,S


# EDA

### 데이터 바꾸기

In [12]:
ori_train.drop('passengerid', axis=1, inplace=True)
ori_train
ori_train.head()

,survived,pclass,name,gender,age,sibsp,parch,ticket,fare,cabin,embarked
0,0,2,"Wheeler, Mr. Edwin Frederick""""",male,NaN,0,0,SC/PARIS 2159,12.8750,NaN,S
1,0,3,"Henry, Miss. Delia",female,NaN,0,0,382649,7.7500,NaN,Q
2,1,1,"Hays, Mrs. Charles Melville (Clara Jennings Gr...",female,52.0,1,1,12749,93.5000,B69,S
3,1,3,"Andersson, Mr. August Edvard (""Wennerstrom"")",male,27.0,0,0,350043,7.7958,NaN,S
4,0,2,"Hold, Mr. Stephen",male,44.0,1,0,26707,26.0000,NaN,S


In [13]:
ori_test.set_index(['passengerid'], inplace=True) 
print(f'{ori_test.shape}')
ori_test.head()

(393, 10)


,pclass,name,gender,age,sibsp,parch,ticket,fare,cabin,embarked
passengerid,,,,,,,,,,
916,3,"McGowan, Miss. Anna ""Annie""",female,15.0,0,0,330923,8.0292,NaN,Q
917,2,"Pinsky, Mrs. (Rosa)",female,32.0,0,0,234604,13.0000,NaN,S
918,3,"McCarthy, Miss. Catherine Katie""""",female,NaN,0,0,383123,7.7500,NaN,Q
919,3,"Franklin, Mr. Charles (Charles Fardon)",male,NaN,0,0,SOTON/O.Q. 3101314,7.2500,NaN,S
920,1,"Wick, Mrs. George Dennick (Mary Hitchcock)",female,45.0,1,1,36928,164.8667,NaN,S


### 수치형 데이터 분석

In [14]:
df_number = ori_train.select_dtypes(include=np.number)

In [15]:
df_number.columns

Index(['survived', 'pclass', 'age', 'sibsp', 'parch', 'fare'], dtype='object')

In [16]:
df_number.describe() # age가 수가 적음(결측치가 있을 것이다)

,survived,pclass,age,sibsp,parch,fare
count,916.000000,916.000000,736.000000,916.000000,916.000000,916.000000
mean,0.377729,2.292576,29.698370,0.507642,0.361354,32.402710
std,0.485084,0.838675,14.185627,1.044866,0.828054,50.506411
min,0.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,21.000000,0.000000,0.000000,7.895800
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.458300
75%,1.000000,3.000000,38.000000,1.000000,0.000000,30.017700
max,1.000000,3.000000,80.000000,8.000000,9.000000,512.329200


In [112]:
df_number['pclass'].value_counts()

3    495
1    227
2    194
Name: pclass, dtype: int64

In [19]:
df_number[df_number['pclass']==3].mean()

survived     0.280808
pclass       3.000000
age         24.643483
sibsp        0.587879
parch        0.408081
fare        13.566363
dtype: float64

In [20]:
df_number[df_number['pclass']==2].mean()

survived     0.391753
pclass       2.000000
age         29.844262
sibsp        0.386598
parch        0.319588
fare        20.388036
dtype: float64

In [21]:
df_number[df_number['pclass']==1].mean()

survived     0.577093
pclass       1.000000
age         38.697563
sibsp        0.436123
parch        0.295154
fare        83.745613
dtype: float64

### 범주형 데이터 분석

In [22]:
df_object = ori_train.select_dtypes(exclude=np.number)

In [23]:
df_object = df_object.join(df_number['survived']) # survived 추가
df_object['survived'] = df_object['survived'].astype('object') # 데이터 타입 변경
df_object.columns

Index(['name', 'gender', 'ticket', 'cabin', 'embarked', 'survived'], dtype='object')

In [24]:
df_object['gender'].value_counts()

male      589
female    327
Name: gender, dtype: int64

In [25]:
df_object['embarked'].value_counts()

S    645
C    189
Q     81
Name: embarked, dtype: int64

# train_test_split


In [26]:
y = ori_train['survived']
X = ori_train.drop(['survived'], axis=1)

In [27]:
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.3, stratify=ori_train['survived'], random_state=21)
# stratify 는 층을 나누는데 비율로 똑같이 나눠줘라는 뜻
# X_tr , X_te 는 feature와 target칼럼이 있고 Y_tr , Y_te는 target 칼럼만 있음
X_tr.shape, X_te.shape, y_tr.shape, y_te.shape

((641, 10), (275, 10), (641,), (275,))

In [28]:
train = X_tr.copy()
test = X_te.copy()
ori_te = ori_test.copy()

train.shape , test.shape , ori_te.shape

((641, 10), (275, 10), (393, 10))

In [29]:
train.drop(['ticket','cabin','name'], axis=1, inplace=True)

In [30]:
test.drop(['ticket','cabin','name'], axis=1, inplace=True)

In [31]:
ori_te.shape , ori_test.shape

((393, 10), (393, 10))

# *Base ModelV0*

### data processing

#### 결측치 처리

In [32]:
train.isnull().sum()

pclass        0
gender        0
age         129
sibsp         0
parch         0
fare          0
embarked      0
dtype: int64

In [33]:
test.isnull().sum()

pclass       0
gender       0
age         51
sibsp        0
parch        0
fare         0
embarked     1
dtype: int64

In [34]:
ori_te.isnull().sum()

pclass        0
name          0
gender        0
age          83
sibsp         0
parch         0
ticket        0
fare          1
cabin       296
embarked      1
dtype: int64

In [35]:
age_median = train['age'].median()
fare_median = train['fare'].median()
embarked_mode = train['embarked'].mode().values[0]

age_median, fare_median, embarked_mode

(28.0, 14.4, 'S')

In [36]:
train['age'].fillna(age_median, inplace=True)
test['age'].fillna(age_median, inplace=True)
ori_te['age'].fillna(age_median, inplace=True)

train['fare'].fillna(fare_median, inplace=True)
test['fare'].fillna(fare_median, inplace=True)
ori_te['fare'].fillna(fare_median, inplace=True)

train['embarked'].fillna(embarked_mode, inplace=True)
test['embarked'].fillna(embarked_mode, inplace=True)
ori_te['embarked'].fillna(embarked_mode, inplace=True)


### data encoding

In [37]:
from sklearn.preprocessing import OneHotEncoder

In [38]:
train.columns 
# train data 중에서 70퍼인 train (641,8)

Index(['pclass', 'gender', 'age', 'sibsp', 'parch', 'fare', 'embarked'], dtype='object')

In [39]:
enc_cols = ['gender', 'embarked'] # 문자형 데이터
normal_cols = list(set(train.columns) - set(enc_cols))
normal_cols # gender , embarked 를 뺸 있는 list(수치형데이터)

['fare', 'sibsp', 'age', 'parch', 'pclass']

In [40]:
enc = OneHotEncoder()

tmp_tr = pd.DataFrame(
    enc.fit_transform(train[enc_cols]).toarray(), #범주형 encoder 적용시켜서 temp_tr에 저장
    columns = enc.get_feature_names_out() # feature name을 뽑아냄 
)  
enc_tr = pd.concat(
    [train[normal_cols].reset_index(drop=True), tmp_tr.reset_index(drop=True)]
    , axis=1
) 
# 수치형 데이터(train[normal_cols]) + encoding 된 문자형 데이터(tmp_tr) 를 합쳐서 enc_tr



# test (ori_te 에서 나온 te  -> 모델의 학습을 평가(잘했는지 못했는지))
tmp_te = pd.DataFrame(
    enc.transform(test[enc_cols]).toarray(), 
    columns = enc.get_feature_names_out()
)
enc_te = pd.concat(
    [test[normal_cols].reset_index(drop=True), tmp_te.reset_index(drop=True)]
    , axis=1
)


# ori_test
tmp_te = pd.DataFrame(
    enc.transform(ori_te[enc_cols]).toarray(), 
    columns = enc.get_feature_names_out()
)
enc_ori_te = pd.concat(
    [ori_te[normal_cols].reset_index(drop=True), tmp_te.reset_index(drop=True)]
    , axis=1
)
print(f'before: {train.shape} / {test.shape} / {ori_te.shape}')
print(f'after: {enc_tr.shape} / {enc_te.shape} / {enc_ori_te.shape}') # gender가 두개로 , embarked가 3개로 

before: (641, 7) / (275, 7) / (393, 10)
after: (641, 10) / (275, 10) / (393, 10)


### Training

In [41]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [42]:
modelV0 = RandomForestClassifier(random_state=21)

print(f'{enc_tr.shape} / {y_tr.shape}')
modelV0.fit(enc_tr, y_tr) # 적합 시킨다 

(641, 10) / (641,)


RandomForestClassifier(random_state=21)

### Evaluation

In [43]:
score_tr = modelV0.score(enc_tr, y_tr) # 인코딩된 train 데이터로 Y_tr 타겟팅 
score_te = modelV0.score(enc_te, y_te) # 인코딩된 test 데이터로 Y_te 타겟팅

score_tr, score_te  

(0.982839313572543, 0.8145454545454546)

In [44]:
from sklearn.metrics import roc_curve, auc 

y_pred = modelV0.predict_proba(enc_te)[:,1] # 인코딩된 모델을 예측함
# 학습률 평가 
fpr, tpr, thresholds = roc_curve(y_te,y_pred)
auc_te = auc(fpr, tpr)
print(f'model: {auc_te}')

model: 0.8926282051282051


In [45]:
ori_te_pred = modelV0.predict_proba(enc_ori_te)[:,1] # enc_ori_te (실제 test 데이터에 적용을 시킴)

In [46]:
df_feature_importances = pd.DataFrame(modelV0.feature_importances_, enc_tr.columns).sort_values(by=[0], ascending=False).reset_index()

print(f'{df_feature_importances.shape}')
df_feature_importances

(10, 2)


,index,0
0,gender_male,0.255023
1,fare,0.206527
2,age,0.188579
3,gender_female,0.182717
4,pclass,0.061065
5,sibsp,0.044225
6,parch,0.032670
7,embarked_S,0.011767
8,embarked_C,0.009472
9,embarked_Q,0.007954


In [47]:
args.results.append(
    {
        'model': 'modelV0',
        'score_tr': score_tr,
        'score_te': score_te,
        'auc_te': auc_te,
        'ori_te_pred': ori_te_pred,
        'len_features': X_tr.shape[1],
        'feaute_importances': list(df_feature_importances['index'].values[:X_tr.shape[1]]),
        'create_dt': '0330'
    }
)

# *Base ModelV1*

In [48]:
train = X_tr.copy() 
test = X_te.copy()
ori_te = ori_test.copy()

train.shape, test.shape, ori_te.shape

((641, 10), (275, 10), (393, 10))

### Data preprocessing

In [49]:
print(f'before: {train.shape} / {test.shape}')
drop_cols = ['name', 'ticket', 'cabin']

train.drop(drop_cols, axis=1, inplace=True)
test.drop(drop_cols, axis=1, inplace=True)
ori_te.drop(drop_cols, axis=1, inplace=True)


print(f'after: {train.shape} / {test.shape}')

before: (641, 10) / (275, 10)
after: (641, 7) / (275, 7)


#### 결측치 처리

In [50]:
age_median = train['age'].median()
fare_median = train['fare'].median()
embarked_mode = train['embarked'].mode().values[0]

age_median, fare_median, embarked_mode

(28.0, 14.4, 'S')

In [51]:
train['age'].fillna(age_median, inplace=True)
test['age'].fillna(age_median, inplace=True)
ori_te['age'].fillna(age_median, inplace=True)

train['fare'].fillna(fare_median, inplace=True)
test['fare'].fillna(fare_median, inplace=True)
ori_te['fare'].fillna(fare_median, inplace=True)

train['embarked'].fillna(embarked_mode, inplace=True)
test['embarked'].fillna(embarked_mode, inplace=True)
ori_te['embarked'].fillna(embarked_mode, inplace=True)

train.isnull().sum().sum(), test.isnull().sum().sum(), ori_te.isnull().sum().sum()

(0, 0, 0)

### data encoding

In [52]:
enc_cols = ['gender', 'embarked']
normal_cols = list(set(train.columns) - set(enc_cols))
normal_cols

['fare', 'sibsp', 'age', 'parch', 'pclass']

In [53]:
enc = OneHotEncoder()

tmp_tr = pd.DataFrame(
    enc.fit_transform(train[enc_cols]).toarray(), #범주형 encoder 적용시켜서 temp_tr에 저장
    columns = enc.get_feature_names_out() # feature name을 뽑아냄 
)  
enc_tr = pd.concat(
    [train[normal_cols].reset_index(drop=True), tmp_tr.reset_index(drop=True)]
    , axis=1
) 
# 수치형 데이터(train[normal_cols]) + encoding 된 문자형 데이터(tmp_tr) 를 합쳐서 enc_tr



# test (ori_te 에서 나온 te  -> 모델의 학습을 평가(잘했는지 못했는지))
tmp_te = pd.DataFrame(
    enc.transform(test[enc_cols]).toarray(), 
    columns = enc.get_feature_names_out()
)
enc_te = pd.concat(
    [test[normal_cols].reset_index(drop=True), tmp_te.reset_index(drop=True)]
    , axis=1
)


# ori_test
tmp_te = pd.DataFrame(
    enc.transform(ori_te[enc_cols]).toarray(), 
    columns = enc.get_feature_names_out()
)
enc_ori_te = pd.concat(
    [ori_te[normal_cols].reset_index(drop=True), tmp_te.reset_index(drop=True)]
    , axis=1
)
print(f'before: {train.shape} / {test.shape} / {ori_te.shape}')
print(f'after: {enc_tr.shape} / {enc_te.shape} / {enc_ori_te.shape}')
# gender가 두개로 , embarked가 3개로 

before: (641, 7) / (275, 7) / (393, 7)
after: (641, 10) / (275, 10) / (393, 10)


### Scaler

In [54]:
enc_tr.columns

Index(['fare', 'sibsp', 'age', 'parch', 'pclass', 'gender_female',
       'gender_male', 'embarked_C', 'embarked_Q', 'embarked_S'],
      dtype='object')

In [55]:
scaling_cols = ['age', 'fare']
not_scaling_cols = list(set(enc_tr.columns) - set(scaling_cols))
not_scaling_cols 

['sibsp',
 'embarked_C',
 'gender_female',
 'embarked_S',
 'embarked_Q',
 'parch',
 'pclass',
 'gender_male']

In [56]:
from sklearn.preprocessing import StandardScaler

In [57]:
std = StandardScaler()

_scaled_tr = std.fit_transform(enc_tr[scaling_cols])
_scaled_te = std.transform(enc_te[scaling_cols])
_scaled_ori_te = std.transform(enc_ori_te[scaling_cols])

In [58]:
print(f'before: {enc_tr.shape} / {enc_te.shape} / {enc_ori_te.shape}')
# train
tmp_tr = pd.DataFrame(
    _scaled_tr, 
    columns = scaling_cols
)
scaled_tr = pd.concat(
    [enc_tr[not_scaling_cols].reset_index(drop=True), tmp_tr.reset_index(drop=True)]
    , axis=1
).reset_index(drop=True)

# test
tmp_te = pd.DataFrame(
    _scaled_te, 
    columns = scaling_cols
)
scaled_te = pd.concat(
    [enc_te[not_scaling_cols].reset_index(drop=True), tmp_te.reset_index(drop=True)]
    , axis=1
).reset_index(drop=True)

# ori_test
tmp_te = pd.DataFrame(
    _scaled_ori_te, 
    columns = scaling_cols
)
scaled_ori_te = pd.concat(
    [enc_ori_te[not_scaling_cols].reset_index(drop=True), tmp_te.reset_index(drop=True)]
    , axis=1
).reset_index(drop=True)


print(f'after: {scaled_tr.shape} / {scaled_te.shape} / {scaled_ori_te.shape}')

before: (641, 10) / (275, 10) / (393, 10)
after: (641, 10) / (275, 10) / (393, 10)


### Training

In [59]:
from sklearn.ensemble import RandomForestClassifier

modelV1 = RandomForestClassifier(random_state=21)

modelV1.fit(scaled_tr, y_tr)

RandomForestClassifier(random_state=21)

### Evaluation

In [60]:
score_tr = modelV1.score(scaled_tr, y_tr)
score_te = modelV1.score(scaled_te, y_te) 

score_tr, score_te  

(0.982839313572543, 0.8254545454545454)

In [61]:
from sklearn.metrics import roc_curve, auc 

y_pred = modelV1.predict_proba(scaled_te)[:,1]
fpr, tpr, thresholds = roc_curve(y_te,y_pred)
auc_te = auc(fpr, tpr)
print(f'model: {auc_te}')

model: 0.893921502474134


In [62]:
ori_te_pred = modelV1.predict_proba(scaled_ori_te)[:,1]

In [63]:
df_feature_importances = pd.DataFrame(modelV1.feature_importances_, scaled_tr.columns).sort_values(by=[0], ascending=False).reset_index()
print(f'{df_feature_importances.shape}')

(10, 2)


In [64]:
args.results.append(
    {
        'model': 'modelV1',
        'score_tr': score_tr,
        'score_te': score_te,
        'auc_te': auc_te,
        'ori_te_pred': ori_te_pred,
        'len_features': X_tr.shape[1],
        'feaute_importances': list(df_feature_importances['index'].values[:X_tr.shape[1]]),
        'create_dt': '0330'
    }
)

# Base ModelV2

In [65]:
train = X_tr.copy() 
test = X_te.copy()
ori_te = ori_test.copy()

train.shape, test.shape, ori_te.shape

((641, 10), (275, 10), (393, 10))

### Data processing

In [66]:
print(f'before: {train.shape} / {test.shape}')
drop_cols = ['name', 'ticket', 'cabin']

train.drop(drop_cols, axis=1, inplace=True)
test.drop(drop_cols, axis=1, inplace=True)
ori_te.drop(drop_cols, axis=1, inplace=True)

print(f'after: {train.shape} / {test.shape}')

before: (641, 10) / (275, 10)
after: (641, 7) / (275, 7)


#### 결측치 처리

In [67]:
age_median = train['age'].median()
fare_median = train['fare'].median()
embarked_mode = train['embarked'].mode().values[0]

train['age'].fillna(age_median, inplace=True)
test['age'].fillna(age_median, inplace=True)
ori_te['age'].fillna(age_median, inplace=True)

train['fare'].fillna(fare_median, inplace=True)
test['fare'].fillna(fare_median, inplace=True)
ori_te['fare'].fillna(fare_median, inplace=True)

train['embarked'].fillna(embarked_mode, inplace=True)
test['embarked'].fillna(embarked_mode, inplace=True)
ori_te['embarked'].fillna(embarked_mode, inplace=True)

train.isnull().sum().sum(), test.isnull().sum().sum(), ori_te.isnull().sum().sum()

(0, 0, 0)

### scaler

In [68]:
scaling_cols = ['age', 'fare']
not_scaling_cols = list(set(enc_tr.columns) - set(scaling_cols))

from sklearn.preprocessing import RobustScaler

rbs = RobustScaler()
_scaled_tr = rbs.fit_transform(enc_tr[scaling_cols])
_scaled_te = rbs.transform(enc_te[scaling_cols])
_scaled_ori_te = rbs.transform(enc_ori_te[scaling_cols])

print(f'before: {enc_tr.shape} / {enc_te.shape}')
# train
tmp_tr = pd.DataFrame(
    _scaled_tr, 
    columns = scaling_cols
)
scaled_tr = pd.concat(
    [enc_tr[not_scaling_cols].reset_index(drop=True), tmp_tr.reset_index(drop=True)]
    , axis=1
).reset_index(drop=True)
# test
tmp_te = pd.DataFrame(
    _scaled_te, 
    columns = scaling_cols
)
scaled_te = pd.concat(
    [enc_te[not_scaling_cols].reset_index(drop=True), tmp_te.reset_index(drop=True)]
    , axis=1
).reset_index(drop=True)
# ori_test
tmp_te = pd.DataFrame(
    _scaled_ori_te, 
    columns = scaling_cols
)
scaled_ori_te = pd.concat(
    [enc_ori_te[not_scaling_cols].reset_index(drop=True), tmp_te.reset_index(drop=True)]
    , axis=1
).reset_index(drop=True)

print(f'after: {scaled_tr.shape} / {scaled_te.shape}')

before: (641, 10) / (275, 10)
after: (641, 10) / (275, 10)


In [69]:
scaled_tr.shape, scaled_te.shape, scaled_ori_te.shape

((641, 10), (275, 10), (393, 10))

### Training

In [70]:
from sklearn.tree import DecisionTreeClassifier

modelV2 = RandomForestClassifier(random_state=21)

modelV2.fit(scaled_tr, y_tr)

RandomForestClassifier(random_state=21)

### Evaluation

In [71]:
score_tr = modelV2.score(scaled_tr, y_tr)
score_te = modelV2.score(scaled_te, y_te) 

score_tr, score_te  

(0.982839313572543, 0.8218181818181818)

In [72]:
from sklearn.metrics import roc_curve, auc 

y_pred = modelV2.predict_proba(scaled_te)[:,1]
fpr, tpr, thresholds = roc_curve(y_te,y_pred)
auc_te = auc(fpr, tpr)
print(f'model: {auc_te}')

model: 0.8947087269455691


In [73]:
ori_te_pred = modelV2.predict_proba(scaled_ori_te)[:,1]

In [74]:
df_feature_importances = pd.DataFrame(modelV2.feature_importances_, scaled_tr.columns).sort_values(by=[0], ascending=False).reset_index()
print(f'{df_feature_importances.shape}')

(10, 2)


In [75]:
args.results.append(
    {
        'model': 'modelV2',
        'score_tr': score_tr,
        'score_te': score_te,
        'auc_te': auc_te,
        'ori_te_pred': ori_te_pred,
        'len_features': X_tr.shape[1],
        'feaute_importances': list(df_feature_importances['index'].values[:X_tr.shape[1]]),
        'create_dt': '0330'
    }
)

# Submission

In [76]:
df_results = pd.DataFrame(args.results).sort_values(by=['auc_te'], ascending=False)
df_results

,model,score_tr,score_te,auc_te,ori_te_pred,len_features,feaute_importances,create_dt
2,modelV2,0.982839,0.821818,0.894709,"[0.74, 0.99, 1.0, 0.003333333333333333, 0.91, ...",10,"[gender_female, gender_male, fare, age, pclass...",0330
1,modelV1,0.982839,0.825455,0.893922,"[0.74, 0.99, 1.0, 0.003333333333333333, 0.91, ...",10,"[gender_female, gender_male, fare, age, pclass...",0330
0,modelV0,0.982839,0.814545,0.892628,"[0.72, 0.98, 1.0, 0.003333333333333333, 0.96, ...",10,"[gender_male, fare, age, gender_female, pclass...",0330


In [77]:
submission = pd.read_csv(args.default_submission_csv)
submission.head(2)

,passengerid,survived
0,916,0.5
1,917,0.5


In [97]:
submission['survived'] = df_results.loc[2, ['ori_te_pred']].values[0] # 인덱스 2번에 헤당하는 ori_te_pred 뽑고 values는 array를 하나 풀어주기 위한 용도
print(f'{submission.isnull().sum().sum()}')
submission.head()

0


,passengerid,survived
0,916,0.740000
1,917,0.990000
2,918,1.000000
3,919,0.003333
4,920,0.910000


In [98]:
submission.to_csv(args.submission_csv, header=True, index=False) # 앞에서 지정한 submission_0330.csv로 저장된다

## 기존 결과 저장

In [99]:
args.save_results

'C:\\Users\\User\\Desktop\\model_results.csv'

In [100]:
df_results.columns

Index(['model', 'score_tr', 'score_te', 'auc_te', 'ori_te_pred',
       'len_features', 'feaute_importances', 'create_dt'],
      dtype='object')

In [101]:
df_results.drop(['ori_te_pred'], axis=1, inplace=True) 

In [107]:
df_results.head()

,model,score_tr,score_te,auc_te,len_features,feaute_importances,create_dt
2,modelV2,0.982839,0.821818,0.894709,10,"[gender_female, gender_male, fare, age, pclass...",0330
1,modelV1,0.982839,0.825455,0.893922,10,"[gender_female, gender_male, fare, age, pclass...",0330
0,modelV0,0.982839,0.814545,0.892628,10,"[gender_male, fare, age, gender_female, pclass...",0330
